In [1]:
import sys
sys.path.insert(0, '../util')

from layer import *
from plot import *

from __future__ import division
import random
import math
import numpy as np
import pandas as pd
import time
from numpy.linalg import norm

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X = MinMaxScaler().fit_transform(load_iris().data)
y = np.vectorize(lambda x: 1 if(x>0) else 0)(load_iris().target).astype(np.float64)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state=10, stratify=y)
X_train = np.transpose(X_train)
X_test = np.transpose(X_test)
y_train = y_train.reshape(1,-1)
y_test = y_test.reshape(1,-1)

In [2]:
input_size = X.shape[1]
computational_layer = 5
output_layer = 1
dimensions = calculate_dimensions(input_size, computational_layer, output_layer)

def position_check(new_position):
    new_position = np.vectorize(lambda z: Wildebeest.pmax if z > Wildebeest.pmax else float(z)) (new_position)
    new_position = np.vectorize(lambda z: Wildebeest.pmin if z < Wildebeest.pmin else float(z)) (new_position)
    return new_position

In [3]:
class Wildebeest:
    global dimensions
    global_best_position = np.random.random((dimensions,))
    global_best_cost, _ = forward_pass_weights_bin(global_best_position,X_train,y_train,input_size,computational_layer,output_layer)
    pmax = 10
    pmin = -10
    
    def __init__(self, dimensions):
        self.position = np.random.random((dimensions,))
        self.cost, _ = forward_pass_weights_bin(self.position,X_train,y_train,input_size,computational_layer,output_layer)
        self.dimensions = dimensions
        self.velocity = np.random.random((dimensions,))
        self.pmax = 10
        self.pmin = -10
        
        self.alpha_1 = 0.9 #Local movement
        self.beta_1 = 0.3  #Local movement
        self.alpha_2 = 0.2 #Global movement
        self.beta_2 = 0.8  #Global movement
        
        self.delta_w = float(dimensions/8.0) #Threshold for nearness
        self.delta_c = 2 #Threshold for avoiding crowds
        
        self.learning_rate = 0.2
        self.n_steps = 4
        self.n_e = 4
        
        self.best_position = self.position
        self.best_cost = self.cost
        
    def update_params(self, position, cost):
        if(cost < Wildebeest.global_best_cost):
            global_best_position = position
            global_best_cost = cost
            
        if(cost < self.cost):
            self.best_position = position
            self.best_cost = cost
            
    def take_steps(self):
        best_cost = self.cost
        best_position = self.position
        for _ in range(self.n_steps):
            new_random_velocity = -1.0 + 2.0*np.random.random((self.dimensions,));
            new_random_position = position_check(self.position + random.random()*new_random_velocity/norm(new_random_velocity))
            new_cost, _ = forward_pass_weights_bin(new_random_position,X_train,y_train,input_size,computational_layer,output_layer)
            if(new_cost<best_cost):
                best_cost = new_cost
                best_position = new_random_position
        return best_position
    
    def update_position(self, new_position):
        self.position = position_check(self.alpha_1 * new_position + self.beta_1 * (self.position - new_position))
        self.cost, _ = forward_pass_weights_bin(self.position,X_train,y_train,input_size,computational_layer,output_layer)
        self.update_params(self.position, self.cost)
        
    def herd_instinct(self, random_wildebeest):
        if(random_wildebeest.best_cost < self.best_cost and random.random() < 0.3):
            self.position = position_check(self.alpha_2 * self.best_position + self.beta_2 * random_wildebeest.best_position)
            self.cost, _ = forward_pass_weights_bin(self.position,X_train,y_train,input_size,computational_layer,output_layer)
            self.update_params(self.position, self.cost)

    def feed_information(self, worst_position):

        if(norm(self.best_position-worst_position) < self.delta_w):
            new_random_velocity = -1 + 2*np.random.random((self.dimensions,));
            self.position = position_check(self.best_position + random.random()*10*(self.pmax-self.pmin)*new_random_velocity/norm(new_random_velocity))
            self.cost, _ = forward_pass_weights_bin(self.position,X_train,y_train,input_size,computational_layer,output_layer)
            self.update_params(self.position, self.cost)
        
        if(norm(Wildebeest.global_best_position - self.position) < self.delta_c and norm(Wildebeest.global_best_position - self.position) > 1):
            self.position = position_check(Wildebeest.global_best_position + 0.0001*np.random.random((self.dimensions,)))
            self.cost, _ = forward_pass_weights_bin(self.position,X_train,y_train,input_size,computational_layer,output_layer)
            
            if(self.cost < Wildebeest.global_best_cost):
                Wildebeest.global_best_position = self.position
                Wildebeest.global_best_cost = self.cost
        
        for _ in range(self.n_e):
            velocity = -1 + 2*np.random.random((self.dimensions,))
            position = position_check(Wildebeest.global_best_position + 0.1 * (velocity/norm(velocity)))
            cost, _ = forward_pass_weights_bin(position,X_train,y_train,input_size,computational_layer,output_layer)
            
            if(cost < Wildebeest.global_best_cost):
                Wildebeest.global_best_position = position
                Wildebeest.global_best_cost = cost


In [4]:
n_iterations = 10
n_population = 80

agents = [Wildebeest(dimensions = dimensions) for x in range(n_population)]
temp = []
cost_list = []

In [5]:
start = time.time()
for _ in range(n_iterations):

    for agent in agents:
        step = agent.take_steps()
        agent.update_position(step)

    for agent in agents:
        random_agent = agents[random.randint(1,n_population)-1]
        agent.herd_instinct(random_agent)

    for agent in agents:
        temp.append([agent.position,agent.cost])
    worst_position = sorted(temp, key = lambda x: x[1])[-1][0]

    for agent in agents:
        agent.feed_information(worst_position)

    cost_list.append(Wildebeest.global_best_cost)

end = time.time()
print(end-start)

cost, output = forward_pass_weights_bin(Wildebeest.global_best_position,X_train,y_train,input_size, computational_layer, output_layer)
print(cost)
print(reg_cost(y_train, output))
print(mae(y_train, output))

2.6279401779174805
-6.666624712254298e-11
1.2972518647511026e-22
1.549655494514044e-12
